In [33]:
%run functions.py

In [25]:
df1.Element.unique()

array(['Production', 'Import Quantity', 'Stock Variation',
       'Export Quantity', 'Domestic supply quantity', 'Seed', 'Losses',
       'Processing', 'Other uses (non-food)', 'Food',
       'Food supply quantity (kg/capita/yr)',
       'Food supply (kcal/capita/day)',
       'Protein supply quantity (g/capita/day)',
       'Fat supply quantity (g/capita/day)', 'Feed',
       'Producer Price (LCU/tonne)', 'Producer Price (SLC/tonne)',
       'Producer Price (USD/tonne)',
       'Producer Price Index (2014-2016 = 100)', 'Tourist consumption',
       'Residuals', 'Food supply quantity (tonnes)',
       'Food supply quantity (g/capita/day)', 'Area harvested', 'Yield',
       'Gross Production Value (constant 2014-2016 1000 I$)',
       'Gross Production Value (current million SLC)',
       'Gross Production Value (constant 2014-2016 million SLC)',
       'Gross Production Value (current million US$)',
       'Gross Production Value (constant 2014-2016 million US$)'],
      dtype=object)

# Umbrella crops

In the ScaleWays project, we aim to identify the most suitable places for extending rice production in the extended Lake Victoria Basin. But before, we need to identify which food and cash crops can be more suitable and/or profitable in the region. We assume that farmers are rational decision makers that will allocate their land to the most suitable+profitable crop. To identify these food and cash crop, hereafter 'potential competitors', we reviewed the FAO 'Food Balances' and 'Food price indices'. 

Different criteria can be used for rank these 'potential competitors'. If our interest is to ensure to include in the set of 'potential competitors', we should use the indicator of 'Food' that represents the "food avaiable for human consumption". Yet, 'Food' may overlook crops of commercial importance, such as Tea and Coffee. In that case, 'GrossProductionValue' will be a better indicator, but we don't have data for Uganda. If our interest is to select the crops based on the extent their are currently produced, 'Production' and 'Area harvested'

The potential competitor crops for rice could be:



You should first select among these variables: 'Food', 'Production', 'FoodPerCapita', 'FoodSelfSufficiency'
these areas: 'LVB', 'RWA', 'UGA', 'KEN', and 'TZA?
And then decide how many crops yuou want to get. 

In [37]:
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


df = pd.read_csv('FAOSTAT_downloaded20210119.csv')
foods = pd.read_csv('foods.csv')


#Bind old (-2013) and new time series (2014-): Food Balances and Producer prices
df.Domain.replace('Food Balances (old methodology and population)', 'FoodBalances', inplace=True)
df.Domain.replace('New Food Balances', 'FoodBalances', inplace=True)
df.Domain.replace('Producer Prices', 'ProducerPrices', inplace=True)
df.Domain.replace('Producer Prices (old series)', 'ProducerPrices', inplace=True)
df.Domain.replace('Food Supply - Crops Primary Equivalent', 'FoodSupply', inplace=True)
df.Domain.replace('Crops', 'CropProduction', inplace=True)
df.Domain.replace('Value of Agricultural Production', 'ValueAgProduction', inplace=True)

#merge with dictionary including "Food groups". Category 2 based on HDDS.
#if you want to filter out non-food, processed foods and beverages, select 'Crop' and 'CropProducts'
df = df.merge(foods, how='left', on='Item')

In [41]:

df1 = df[(df.Domain == 'FoodBalances') &
         ((df.Category1 == 'Crop')|(df.Category1 == 'CropProducts')) &
         (df.Year >= 1991) & (df.Year <= 2019)]
df1 = df1[['Domain', 'Area', 'Year', 'Element', 'Value', 'name']]
df1 = df1.rename(columns={'name' : 'Item'})

a = df1[(df1.Element == 'Production') ].groupby(['Item', 'Area']).mean().reset_index()
a = a.rename(columns={'Value':'Production'})
b = df1[ (df1.Element == 'Food')].groupby(['Item', 'Area']).mean().reset_index()
b = b.rename(columns={'Value':'Food'})
d = df1[(df1.Element == 'Import Quantity')].groupby(['Item', 'Area']).mean().reset_index()
d = d.rename(columns={'Value':'Import'})
e = df1[(df1.Element == 'Export Quantity')].groupby(['Item', 'Area']).mean().reset_index()
e = e.rename(columns={'Value':'Export'})
f = df1[(df1.Element == 'Domestic supply quantity')].groupby(['Item', 'Area']).mean().reset_index()
f = f.rename(columns={'Value':'DomesticSupply'})
g = df1[(df1.Element == 'Food supply quantity (kg/capita/yr)')].groupby(['Item', 'Area']).mean().reset_index()
g = g.rename(columns={'Value':'FoodPerCapita'})

df3 = df[(df.Domain == 'ProducerPrices') & 
         ((df.Category1 == 'Crop')|(df.Category1 == 'CropProducts')) &
         (df.Year >= 1991) & (df.Year <= 2019)] 
df3 = df3[['Domain', 'Area', 'Year', 'Element', 'Value', 'name']]
df3 = df3.rename(columns={'name' : 'Item'})
h = df3[(df3.Element == 'Producer Price Index (2014-2016 = 100)') ].groupby(['Item', 'Area']).mean().reset_index()
h = h.rename(columns = {'Value':'PriceIndex'})

df4 = df[(df.Domain == 'ValueAgProduction') & 
         ((df.Category1 == 'Crop')|(df.Category1 == 'CropProducts')) &
         (df.Year >= 1991) & (df.Year <= 2019)] 
df4 = df4[['Domain', 'Area', 'Year', 'Element', 'Value', 'name']]
df4 = df4.rename(columns={'name' : 'Item'})
i = df4[(df4.Element == 'Gross Production Value (constant 2014-2016 million US$)') ].groupby(['Item', 'Area']).mean().reset_index()
i = i.rename(columns = {'Value':'GrossProductionValue'})

df4 = df[(df.Domain == 'CropProduction') & 
         ((df.Category1 == 'Crop')|(df.Category1 == 'CropProducts')) &
         (df.Year >= 1991) & (df.Year <= 2019)] 
df4 = df4[['Domain', 'Area', 'Year', 'Element', 'Value', 'name']]
df4 = df4.rename(columns={'name' : 'Item'})
j = df4[(df4.Element == 'Area harvested') ].groupby(['Item', 'Area']).mean().reset_index()
j = j.rename(columns = {'Value':'Area harvested'})

df2 = a.merge(b, how='outer', on=['Item', 'Area', 'Year'])
df2 = df2.merge(g, how= 'outer', on=['Item', 'Area', 'Year'])
df2 = df2.merge(h, how= 'outer', on=['Item', 'Area', 'Year'])
df2 = df2.merge(i, how= 'outer', on=['Item', 'Area', 'Year'])
df2 = df2.merge(j, how= 'outer', on=['Item', 'Area', 'Year'])

df2['FoodSelfSufficiency'] = df2.Production / df2.Food  *100
df2['FoodSelfSufficiency'] = df2['FoodSelfSufficiency'].round(1)
df2['FoodPerCapita'] = df2['FoodPerCapita'].round(1)
df2['PriceIndex'] = df2['PriceIndex'].round(1)
df2['GrossProductionValue'] = df2['GrossProductionValue'].round(1)
df2['Production'] = df2['Production'].round(1)
df2['Food'] = df2['Food'].round(1)


In [39]:
potentialCompetitor('Production', 'KEN', 20)

,index,TopCrops,Area,Production,Food,FoodPerCapita,PriceIndex,GrossProductionValue,Area harvested,FoodSelfSufficiency
1,127,Sugar cane,KEN,4987.6,669.7,17.0,72.3,167.8,62.1,744.8
2,55,Maize and products,KEN,2879.1,2975.2,83.1,NaN,NaN,NaN,96.8
3,101,Potatoes and products,KEN,1544.0,1311.8,34.9,NaN,NaN,NaN,117.7
4,3,Bananas,KEN,1163.4,989.0,27.3,76.3,483.5,71.1,117.6
5,134,Sweet potatoes,KEN,639.8,587.8,15.6,57.4,220.5,55.9,108.8
6,15,Cassava and products,KEN,621.5,602.0,16.8,NaN,NaN,NaN,103.2
7,11,Beans,KEN,457.0,379.1,10.2,53.6,18.0,5.1,120.5
8,93,Pineapples and products,KEN,382.6,244.0,7.4,NaN,NaN,NaN,156.8
9,142,Tomatoes and products,KEN,357.4,336.1,9.0,NaN,NaN,NaN,106.3
10,138,Tea,KEN,321.9,41.5,1.0,65.6,818.0,155.5,775.6


In [40]:
graphCompetitors('Food', 10)

TypeError: graphCompetitors() missing 1 required positional argument: 'number'

# Expansion of rice production in East Africa, and its implications on regional food security. 

- East African production is not sufficiency is 


## Changing dietary patterns in East Africa

Diets in East Africa are as heterogeneous as its agroecological zones. Sorghum and millet are widely cultivated and consumed in the semi-arid and arid regions, while platains and potatoes are cultivated in the humid highlands (see main food crops grown in Uganda in Figure 1). Maize, cassava, sweet potatoes, groundnuts, dark-green leaves, and rice are also important food crops in the region. Traditionally, cereal and grains were consumed in porridge ('uji' or 'ugali')  or in stews, and legumes, vegetables, and dark-green leaves were served as suplementary items. Cassava was seen as a reserve food, while meat was consumed during ceremonies or celebrations (Vorster, 1971). 

<br>
<div align="center">  <strong> Figure 1.   Main food crops in Uganda </strong> </div>

<div align="center"> <img src="images\StapleCropsUganda.png"
     alt="Markdown Monster icon"
     style="float: 5px; width:400px;"
     legend = "yep"/> </div>
     


<div align="center">  <em> Source: Uganda's Annual Agriculture Survey, 2018. </em> </div>
    
<br>    
However, dietary patterns have rapidly changed in the last 50 years. Traditional and coarse grains have been replaced by rice and wheat (Kennedy \& Reardon, 1994), consumption of processed foods and sugar have increased (Haggblade et al., 2016), and the amount of fruits and vegetables consumed has been insufficient to cover the micronutrient needs of the population (GNR, 2020). The following 4 figures show a decrease in the consumption of 'traditional' staples -sorghum, cassava, maize, and platains- and an increase in the consumption of rice in Kenya, Rwanda, Tanzania and Uganda. 


### Uganda
Rice in Uganda is sold through three main channels: public markets, small grocery stores, and supermarkets. These channels are targeted to very differetn type of consumers. Rice is sold in packages, mainly in small stores and supermarkets, and loose in public markets. Despite the different varieties of rice that are produced in the country, in the market, rice is branded based on its origin, domestic and foreign productiction, not its variety. 

    
Rapid urbanization and rural-to-urban migration, mordernization of food retailing, and increases in income have been correlated with these dietary changes (Haggblade et al., 2016). A comparison of the 2005 and 2015 expenditure share in the food basket in Kenya showed a larger increase in consumption of rice in urban areas than in rural areas in Kenya (KIHBS, 2015). A similar trend is observed when comparing income groups; rice is consumed more by high-income clases since traditional crops, such as sorghum, are considered “inferior” foods (Minot, 2010). 

These dietary changes have negative impacts on public health. In African countries that have experienced the nutritional transition earlier than in East Africa, e.g., South Africa, it has been observed an increase in overweight and obesity, micronutrient deficiencies, and non-communicable diseases, such as diabetes and cadiovascular diseases (Steyn \& Mchiza, 2014). This can be partially explained by the substitution of whole-grains for refined grains and/or of traditional/diverse grains for standard-looking grains. But there are other processes influencing this, such as an increase in consumption of fast food and a decrease in consumption of vegetables and fruits. During the milling process, grains loss some vitamins that are contained in the bran and germ, such as vitamins B. For instance, brown rice and African rice (*Oryza glaberrima*) contain considerably more Thiamin, Riboflavin, Niacin, and Folate than white rice (see Table 1). The same can be observed when comparing whole-wheat with wheat flour. Similarly, yellow maize, which is associated with 'animal feed' and considered as 'poor-people's food' (IPBO, 2017), is richer in Vitamin A than white maize.   


**Table.1   Nutritional composition of main staple crops consumed in East Africa.**
    
|                                |     Rice, white, polished, raw | Rice, brown, raw |     Rice, red native, hulled, raw |     Maize, white, whole   kernel,   dried, raw |     Maize, yellow, whole   kernel,   dried, raw |     Wheat, whole grains, raw |     Wheat flour, white |     Sorghum, whole grain, raw  |     Pearl millet, whole grain,   raw   (with bran)    |     Potato, raw       |     Cassava, tuber, raw                     |     Plantain, ripe, raw |     Sweet potato, pale yellow, raw |     Sweet potato, deep yellow, raw |
|--------------------------------|--------------------------------|------------------|-----------------------------------|------------------------------------------------|-------------------------------------------------|------------------------------|------------------------|--------------------------------|-------------------------------------------------------|-----------------------|---------------------------------------------|-------------------------|------------------------------------|------------------------------------|
|     Scientific name            |     *Oryza sativa*               | *Oryza sativa*     |     *Oryza glaberrima*              |     Zea mays                                   |     Zea mays                                    |     Triticum spp.            |     Triticum spp.      |     Sorghum bicolor            |     Pennisetum glaucum                                |     Solanum tuberosum |     Manihot esculenta/   Manihot utilissima |     Musa paradisica     |     Ipomoea batatas                |     *Ipomea batatas*                 |
|     Edible   conversion factor |     1                          | 1                |     1                             |     1                                          |     1                                           |     1                        |     1                  |     1                          |     1                                                 |     0.84              |     0.84                                    |     0.65                |     0.84                           |     0.84                           |
|     Energy   kcal              |     353                        | 352              |     359                           |     349                                        |     353                                         |     326                      |     351                |     344                        |     364                                               |     80                |     153                                     |     140                 |     115                            |     112                            |
|     Protein   (g)              |     6.1                        | 7.8              |     7.4                           |     9.2                                        |     9                                           |     12.4                     |     10.4               |     10.5                       |     8.8                                               |     1.9               |     1.2                                     |     1.2                 |     1.5                            |     1.5                            |
|     Fat (g)                    |     0.5                        | 2.2              |     2.2                           |     4.1                                        |     4.5                                         |     2.2                      |     1.5                |     3.3                        |     5.8                                               |     0.1               |     0.3                                     |     0.3                 |     0.2                            |     0.3                            |
|     Fibre (g)                  | 0.4                            | 0.4              | 0.4                               |     9.7                                        |     9.7                                         |     10.8                     |     3.2                |     9.9                        |     9                                                 |     1.8               |     1.8                                     |     2.3                 |     3                              |     3                              |
|     Ca    (mg)                 |     11                         | 22               |     38                            |     19                                         |     12                                          |     30                       |     19                 |     24                         |     14                                                |     11                |     43                                      |     7                   |     27                             |     26                             |
|     Fe   (mg)                  |     0.7                        | 1.9              |     2.8                           |     3.1                                        |     3.5                                         |     4.7                      |     2                  |     3.7                        |     7.6                                               |     0.9               |     0.7                                     |     0.9                 |     1.1                            |     1.1                            |
|     Vit A-RAE   (mcg)          |     0                          | 0                |     0                             |     0                                          |     50                                          |     0                        |     0                  |     1                          |     0                                                 |     1                 |     1                                       |     43                  |     3                              |     397                            |
|     Thiamin (mg)               |     0.07                       | 0.38             |     0.34                          |     0.35                                       |     0.33                                        |     0.47                     |     0.28               |     0.36                       |     0.32                                              |     0.08              |     0.04                                    |     0.07                |     0.09                           |     0.09                           |
|     Riboflavin  (mg)           |     0.03                       | 0.07             |     0.1                           |     0.1                                        |     0.15                                        |     0.1                      |     0.1                |     0.16                       |     0.27                                              |     0.12              |     0.05                                    |     0.05                |     0.04                           |     0.05                           |
|     Niacin   (mg)              |     0.4                        | 5.0              |     6.5                           |     2.1                                        |     2.2                                         |     5.6                      |     1.2                |     3.3                        |     2.4                                               |     1.2               |     0.7                                     |     0.7                 |     0.6                            |     0.7                            |
|     Folate   (mcg)             |     20                         | 40               |                                   |     26                                         |     26                                          |     45                       |     24                 |     29                         |     30                                                |     18                |     24                                      |     22                  |     52                             |     52                             |


*Source:* FAO/INFOODS, (2012). Food Composition Table for Western Africa.
 
 
### Growing rice demand

Rice has grown in importance in East African countries, not only as food, but also as an economic opportunity. ... Something about rice consumption.... Rice serves both as a food staple for 70% of farming households and as a cash crop for 80% of these households ()... The share of total calories supplied by rice has been growing steadly since 1961 (Figure #). In the 60's, rice barely contributed to the kcal consumed in Kenya, Uganda, and Rwanda; yet, in 2013, it contributed with 2-5% of the calories consumed per capita in these countries. In Tanzania, where rice can be considered a staple, the share of calories coming from rice has increased from 3.7% in 1961 to 9.2% in 2013. 
    
**Figure 3. Percentage of total calories coming from main staple crops**. 

*Source: FAOSTAT*

Due to its popularity and demand, especially concerning aromatic rice, EAC countries import rice. Although Tanzania is largely self-sufficient and imports merely 140,000 MT rice per annum, other EAC import rice to varying degrees: Kenya imports 0.3 million MT and Rwanda imports 20 MT  (KilimoTrust, 2014a, p. 8). Rice is mainly imported from Asian countries (see Fig. 3), which is prized lower than rice produced in the region. The main rice exporters are Pakistan, Thailand, Tanzania, India and China, who combined exported a total of 1.1 million tons of rice in 2018 -an increase of 30% compared to year 2015 (UN Comtrade, 2020). Yet, trade within the EAC region plays a significant role in covering national demands; for example, Tanzania exported 82,300 t to Uganda in 2018 and 5,100 t to Burundi. Uganda also exported half of its production to other countries, such as Congo and South Sudan (UN Comtrade, 2020).   

...
    
#### Rice self-sufficiency 

Rice self-sufficiency has been rarely achieved in the last years in East African countries, with the exception of Tanzania. Although we have observed an increase in the harvested area and yields of paddy rice, domestic production has not been sufficient to cover the food demand for rice in Uganda, Rwanda, and Kenya as shown in Figure 4. In Rwanda, for instance, the production-consumption ratio (P/C) shows that self-sufficiency was achieved only once in the period between 1961-2013.     

**Figure 4. Rice harvested area, yields, imports and production/consumption ratio.** 